In [1]:
# import models and solvers
from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
from centroidal_plus_double_integrator_kinematics_casadi_model  import CentroidalPlusLegKinematicsCasadiModel
import conf_solo12_trot_step_adjustment_full_kinematics_mpc as conf
from robot_properties_solo.solo12wrapper import Solo12Config
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
import pinocchio as pin
import numpy as np
import utils
import sys

# DDP warm-start
wbd_model = WholeBodyModel(conf)
ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=False, WARM_START=False)
ddp_planner.solve()
ddp_sol = ddp_planner.get_solution_trajectories()
centroidal_warmstart = ddp_sol['centroidal']
q_warmstart = ddp_sol['jointPos']
qdot_warmstart = ddp_sol['jointVel']
x_warmstart = []
u_warmstart = []
rmodel, rdata = conf.rmodel, conf.rdata
for k in range(len(centroidal_warmstart)):
    x_warmstart.append(
        np.concatenate(
            [centroidal_warmstart[k],
             q_warmstart[k], 
             qdot_warmstart[k]]
            )
        )
    u_warmstart.append(np.concatenate([np.zeros(30)]))

# create robot
robot = Solo12Config.buildRobotWrapper()
# load robot in meshcat viewer
viz = pin.visualize.MeshcatVisualizer(
robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print(err)
    sys.exit(0)
viz.loadViewerModel()
# add contact surfaces
s = 0.5*conf.step_adjustment_bound
for i, contacts in enumerate(conf.contact_sequence):
    for contact_idx, contact in enumerate(contacts):
        if contact.ACTIVE:
            t = contact.pose.translation
            # debris box
            if contact.CONTACT == 'FR' or contact.CONTACT == 'FL':
                utils.addViewerBox(
                    viz, 'world/debris'+str(i)+str(contact_idx), 
                    2*s, 2*s, 0., [1., .2, .2, .5]
                    )
            if contact.CONTACT == 'HR' or contact.CONTACT == 'HL':
                utils.addViewerBox(
                    viz, 'world/debris'+str(i)+str(contact_idx),
                    2*s, 2*s, 0., [.2, .2, 1., .5]
                    )
            utils.applyViewerConfiguration(
                viz, 'world/debris'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                )
            utils.applyViewerConfiguration(
                viz, 'world/debris_center'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                ) 
# visualize DDP warm-start
for q_warmstart_k in q_warmstart:
    for i in range(10+int(conf.dt/conf.dt_ctrl)):
        viz.display(q_warmstart_k)

# nominal traj-opt
model_nom = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=False)
solver_nom = CentroidalPlusLegKinematicsAcadosSolver(
    model_nom, x_warmstart, u_warmstart, MPC=True
)
x_nom, u_nom, lqr_gains = solver_nom.solve()

# stochastic traj-opt
model_stoch = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=True)
solver_stoch = CentroidalPlusLegKinematicsAcadosSolver(
    model_stoch, x_warmstart, u_warmstart, MPC=True
)
x_stoch, u_stoch, lqr_gains = solver_stoch.solve()

pybullet build time: May 20 2022 19:44:17


Adding Body
"base_link" connected to "world" through joint "floating_base_joint"
joint type: joint FreeFlyer
joint placement:
  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0

body info: 
  mass: 1.16115
  lever: 0 0 0
  inertia elements (Ixx,Iyx,Iyy,Izx,Izy,Izz): 0.00578574          0  0.0193811          0          0  0.0247612

Adding Body
"FL_SHOULDER" connected to "base_link" through joint "FL_HAA"
joint type: joint REVOLUTE with axis
joint placement:
  R =
1 0 0
0 1 0
0 0 1
  p = 0.1946 0.0875      0

body info: 
  mass: 0.148538
  lever: -0.078707      0.01         0
  inertia elements (Ixx,Iyx,Iyy,Izx,Izy,Izz):  3.024e-05  4.671e-05 0.00041193          0          0 0.00041107

Adding Body
"FL_UPPER_LEG" connected to "FL_SHOULDER" through joint "FL_HFE"
joint type: joint REVOLUTE with axis
joint placement:
  R =
1 0 0
0 1 0
0 0 1
  p =     0 0.014     0

body info: 
  mass: 0.148538
  lever: 1.377e-05 0.0193585 -0.078707
  inertia elements (Ixx,Iyx,Iyy,Izx,Izy,Izz): 0.00041107          0 0.00

  26  2.20095e+02  2.28855e+05  2.21050e+01  1.00000e-09  1.00000e-09  0.0625  0.00000e+00
  27  2.18872e+02  1.99950e+05  2.05785e+01  1.00000e-09  1.00000e-09  0.5000  0.00000e+00
  28  2.18341e+02  5.27752e+04  1.86674e+01  1.00000e-09  1.00000e-09  0.0312  0.00000e+00
  29  2.18069e+02  4.94882e+04  1.75374e+01  1.00000e-09  1.00000e-09  0.0156  0.00000e+00
iter     cost         stop         grad         xreg         ureg       step    ||ffeas||
  30  2.18041e+02  4.79440e+04  1.69837e+01  1.00000e-08  1.00000e-08  0.0078  0.00000e+00
  31  2.14917e+02  4.71911e+04  1.66218e+01  1.00000e-08  1.00000e-08  0.5000  0.00000e+00
  32  2.14484e+02  9.50515e+03  1.07152e+01  1.00000e-08  1.00000e-08  0.0625  0.00000e+00
  33  2.14144e+02  8.35570e+03  9.69983e+00  1.00000e-08  1.00000e-08  0.0625  0.00000e+00
  34  2.12380e+02  7.37158e+03  8.98754e+00  1.00000e-08  1.00000e-08  0.5000  0.00000e+00
  35  2.12271e+02  2.03299e+03  5.63755e+00  1.00000e-08  1.00000e-08  0.0625  0.00000e+00


  inertia elements (Ixx,Iyx,Iyy,Izx,Izy,Izz): 5.70000e-07 0.00000e+00 8.40000e-07 0.00000e+00 0.00000e+00 5.30000e-07

Adding Body
"HR_SHOULDER" connected to "base_link" through joint "HR_HAA"
joint type: joint REVOLUTE with axis
joint placement:
  R =
1.00000e+00 0.00000e+00 0.00000e+00
0.00000e+00 1.00000e+00 0.00000e+00
0.00000e+00 0.00000e+00 1.00000e+00
  p = -1.94600e-01 -8.75000e-02  0.00000e+00

body info: 
  mass: 1.48538e-01
  lever:  7.87070e-02 -1.00000e-02  0.00000e+00
  inertia elements (Ixx,Iyx,Iyy,Izx,Izy,Izz):  3.02400e-05 -4.67100e-05  4.11930e-04  0.00000e+00  0.00000e+00  4.11070e-04

Adding Body
"HR_UPPER_LEG" connected to "HR_SHOULDER" through joint "HR_HFE"
joint type: joint REVOLUTE with axis
joint placement:
  R =
1.00000e+00 0.00000e+00 0.00000e+00
0.00000e+00 1.00000e+00 0.00000e+00
0.00000e+00 0.00000e+00 1.00000e+00
  p =  0.00000e+00 -1.40000e-02  0.00000e+00

body info: 
  mass: 1.48538e-01
  lever: -1.37700e-05 -1.93585e-02 -7.87070e-02
  inertia element

starting RTI preparation phase ...
RTI preparation phase took 0.0037610530853271484 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.04340004920959473 seconds

iter	qp_stat	qp_iter
0	0	0
1	0	13


HOORAY ! found a solution after : 0.047161102294921875  seconds

MPC Iteration 6
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.0060236454010009766 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.031475067138671875 seconds

iter	qp_stat	qp_iter
0	0	0
1	0	13


HOORAY ! found a solution after : 0.03749871253967285  seconds

MPC Iteration 7
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.01641082763671875 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.05196237564086914 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.06837320327758789  seconds

MPC Iteration 8
-------------------

...

starting RTI preparation phase ...
RTI preparation phase took 0.017579078674316406 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.045644521713256836 seconds

iter	qp_stat	qp_iter
0	0	0
1	0	14


HOORAY ! found a solution after : 0.06322360038757324  seconds

MPC Iteration 32
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.01672959327697754 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.04520130157470703 seconds

iter	qp_stat	qp_iter
0	0	0
1	0	14


HOORAY ! found a solution after : 0.06193089485168457  seconds

MPC Iteration 33
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.017398595809936523 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.04543256759643555 seconds

iter	qp_stat	qp_iter
0	0	0
1	0	14


HOORAY ! found a solution after : 0.06283116340637207  seconds

MPC Iteration 34
-------------------

starting RTI preparation phase ...
RTI preparation phase took 0.017365694046020508 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.06018710136413574 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.07755279541015625  seconds

MPC Iteration 53
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.029217004776000977 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.07202792167663574 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.10124492645263672  seconds

MPC Iteration 54
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.02528214454650879 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.08786177635192871 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.1131439208984375  seconds

MPC Iteration 55
---------------------

starting RTI preparation phase ...
RTI preparation phase took 0.01698136329650879 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.04775357246398926 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.06473493576049805  seconds

MPC Iteration 74
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.01709604263305664 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.05101323127746582 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.06810927391052246  seconds

MPC Iteration 75
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.01640176773071289 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.05645871162414551 seconds

iter	qp_stat	qp_iter
0	0	0
1	0	14


HOORAY ! found a solution after : 0.0728604793548584  seconds

MPC Iteration 76
-----------------------

starting RTI preparation phase ...
RTI preparation phase took 0.01575303077697754 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.053148746490478516 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.06890177726745605  seconds

MPC Iteration 95
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.03147006034851074 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.04972100257873535 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.0811910629272461  seconds

MPC Iteration 96
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.05564689636230469 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.05010247230529785 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.10574936866760254  seconds

MPC Iteration 97
----------------------

starting RTI preparation phase ...
RTI preparation phase took 0.01884913444519043 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.05122733116149902 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.07007646560668945  seconds

MPC Iteration 116
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.019054174423217773 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.0514528751373291 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.07050704956054688  seconds

MPC Iteration 117
--------------------------------------------------
starting RTI preparation phase ...
RTI preparation phase took 0.019010543823242188 seconds
starting RTI feedback phase ...
RTI feedback phase took 0.04960489273071289 seconds

iter	qp_stat	qp_iter
0	0	0
1	2	15


HOORAY ! found a solution after : 0.06861543655395508  seconds

MPC Iteration 118
------------------

In [8]:
from centroidal_plus_double_integrator_kinematics_acados_simulator import CentroidalPlusLegKinematicsAcadosSimulator
# run monte-carlo simulations
nb_sims = 100
simulator_nom = CentroidalPlusLegKinematicsAcadosSimulator(model_nom, solver_nom)
# sample addtitive disturbances and pass them to both simulators
w_total = np.zeros((nb_sims, conf.N-1, x_nom.shape[1]))
for sim in range(nb_sims):
    for time_idx in range(conf.N-1):
        w_total[sim, time_idx, :] = simulator_nom.sample_addtitive_uncertainties() 

# simulate nominal MPC
x_sim_nom = simulator_nom.simulate(x_nom[0, :], u_nom, lqr_gains, np.copy(w_total), nb_sims=nb_sims, WITH_DISTURBANCES=True)
violations_nom, ee_positions_nom = simulator_nom.count_constraint_violations(x_sim_nom, WITH_VISUALIZATION=False)
print("number of contact location constraint violations for nominal MPC :", violations_nom)

# simulate stochastic MPC
simulator_stoch = CentroidalPlusLegKinematicsAcadosSimulator(model_stoch, solver_stoch)
x_sim_stoch = simulator_stoch.simulate(x_stoch[0, :], u_stoch, lqr_gains, np.copy(w_total), nb_sims=nb_sims, WITH_DISTURBANCES=True)
violations_stoch, ee_positions_stoch = simulator_stoch.count_constraint_violations(x_sim_stoch, WITH_VISUALIZATION=False)
print("number of contact location constraint violations for stochastic MPC :", violations_stoch)

cc -fPIC -std=c99   -O2 -I/home/agazar/devel/workspace/src/acados/include -I/home/agazar/devel/workspace/src/acados/include/acados -I/home/agazar/devel/workspace/src/acados/include/blasfeo/include -I/home/agazar/devel/workspace/src/acados/include/hpipm/include  -c -o acados_sim_solver_quadruped_centroidal_momentum_plus_leg_kinematics.o acados_sim_solver_quadruped_centroidal_momentum_plus_leg_kinematics.c
cc -fPIC -std=c99   -O2 -I/home/agazar/devel/workspace/src/acados/include -I/home/agazar/devel/workspace/src/acados/include/acados -I/home/agazar/devel/workspace/src/acados/include/blasfeo/include -I/home/agazar/devel/workspace/src/acados/include/hpipm/include  -c -o quadruped_centroidal_momentum_plus_leg_kinematics_model/quadruped_centroidal_momentum_plus_leg_kinematics_expl_ode_fun.o quadruped_centroidal_momentum_plus_leg_kinematics_model/quadruped_centroidal_momentum_plus_leg_kinematics_expl_ode_fun.c
cc -fPIC -std=c99   -O2 -I/home/agazar/devel/workspace/src/acados/include -I/home/

starting monte-carlo simulation nb:  76 .....
starting monte-carlo simulation nb:  77 .....
oh oh ..  FL_FOOT_foot is outside assigned debris at time knot  120
starting monte-carlo simulation nb:  78 .....
starting monte-carlo simulation nb:  79 .....
starting monte-carlo simulation nb:  80 .....
starting monte-carlo simulation nb:  81 .....
starting monte-carlo simulation nb:  82 .....
starting monte-carlo simulation nb:  83 .....
oh oh ..  FL_FOOT_foot is outside assigned debris at time knot  120
starting monte-carlo simulation nb:  84 .....
starting monte-carlo simulation nb:  85 .....
oh oh ..  HR_FOOT_foot is outside assigned debris at time knot  120
starting monte-carlo simulation nb:  86 .....
oh oh ..  HL_FOOT_foot is outside assigned debris at time knot  100
starting monte-carlo simulation nb:  87 .....
starting monte-carlo simulation nb:  88 .....
starting monte-carlo simulation nb:  89 .....
oh oh ..  HR_FOOT_foot is outside assigned debris at time knot  120
starting monte-c

starting monte-carlo simulation nb:  88 .....
starting monte-carlo simulation nb:  89 .....
starting monte-carlo simulation nb:  90 .....
starting monte-carlo simulation nb:  91 .....
starting monte-carlo simulation nb:  92 .....
starting monte-carlo simulation nb:  93 .....
starting monte-carlo simulation nb:  94 .....
starting monte-carlo simulation nb:  95 .....
starting monte-carlo simulation nb:  96 .....
starting monte-carlo simulation nb:  97 .....
starting monte-carlo simulation nb:  98 .....
starting monte-carlo simulation nb:  99 .....
number of contact location constraint violations for stochastic MPC : [0, 0, 0, 0]


In [9]:
from robot_properties_solo.solo12wrapper import Solo12Config
import pinocchio as pin
import utils

# create robot
robot = Solo12Config.buildRobotWrapper()
# load robot in meshcat viewer
viz = pin.visualize.MeshcatVisualizer(
robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print(err)
    sys.exit(0)
viz.loadViewerModel()
# add nominal contact surfaces
s = simulator_nom.debris_half_size
for i, contacts in enumerate(simulator_nom.contact_sequence):
    for contact_idx, contact in enumerate(contacts):
        if contact.ACTIVE:
            t = contact.pose.translation
            # debris box
            if contact.CONTACT == 'FR' or contact.CONTACT == 'FL':
                utils.addViewerBox(
                    viz, 'world/debris'+str(i)+str(contact_idx), 
                    2*s, 2*s, 0., [1., .2, .2, .5]
                    )
            if contact.CONTACT == 'HR' or contact.CONTACT == 'HL':
                utils.addViewerBox(
                    viz, 'world/debris'+str(i)+str(contact_idx),
                    2*s, 2*s, 0., [.2, .2, 1., .5]
                    )
            utils.applyViewerConfiguration(
                viz, 'world/debris'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                )
            utils.applyViewerConfiguration(
                viz, 'world/debris_center'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                )
                
ee_pos_total_nom = []
ee_pos_total_stoch = []
# visualize end-effector trajectories (TODO: visualize trajectories in mujoco with an army of robots)
for sim in range(nb_sims):
    FL_ee_nom = []
    FR_ee_nom = []
    HL_ee_nom = []
    HR_ee_nom = []
    FL_ee_stoch = []
    FR_ee_stoch = []
    HL_ee_stoch = []
    HR_ee_stoch = []
    for traj_length_nom in range(len(ee_positions_nom[sim])):
        FL_ee_nom += [ee_positions_nom[sim][traj_length_nom][0]]
        FR_ee_nom += [ee_positions_nom[sim][traj_length_nom][1]]
        HL_ee_nom += [ee_positions_nom[sim][traj_length_nom][2]]
        HR_ee_nom += [ee_positions_nom[sim][traj_length_nom][3]]
    for traj_length_stoch in range(len(ee_positions_stoch[sim])):
        FL_ee_stoch += [ee_positions_stoch[sim][traj_length_stoch][0]]
        FR_ee_stoch += [ee_positions_stoch[sim][traj_length_stoch][1]]
        HL_ee_stoch += [ee_positions_stoch[sim][traj_length_stoch][2]]
        HR_ee_stoch += [ee_positions_stoch[sim][traj_length_stoch][3]]
    # plot end-effectors trajectories
    utils.addLineSegment(viz, 'ee_trajectory_nom_FL'+str(sim), np.array(FL_ee_nom).astype(np.float32).T, [1,1,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_nom_FR'+str(sim), np.array(FR_ee_nom).astype(np.float32).T, [1,1,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_nom_HL'+str(sim), np.array(HL_ee_nom).astype(np.float32).T, [1,1,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_nom_HR'+str(sim), np.array(HR_ee_nom).astype(np.float32).T, [1,1,0,1])    
    utils.addLineSegment(viz, 'ee_trajectory_stoch_FL'+str(sim), np.array(FL_ee_stoch).astype(np.float32).T, [1,0,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_stoch_FR'+str(sim), np.array(FR_ee_stoch).astype(np.float32).T, [1,0,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_stoch_HL'+str(sim), np.array(HL_ee_stoch).astype(np.float32).T, [1,0,0,1])
    utils.addLineSegment(viz, 'ee_trajectory_stoch_HR'+str(sim), np.array(HR_ee_stoch).astype(np.float32).T, [1,0,0,1])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [4]:
# compute the norm of the difference between the closed-loop contact location and the center of contact surface
contacts_logic_N = simulator_nom.contact_data['contacts_logic']
contacts_position_N = simulator_nom.contact_data['contacts_position'] 
contact_location_nom_total   = [] 
contact_location_stoch_total = [] 
contact_surface_location_nom_total = []
contact_surface_location_stoch_total = []
norm_contact_location_deviation_nom_total = []
norm_contact_location_deviation_stoch_total = []
for sim in range(nb_sims):
    contact_location_nom_per_traj = []
    contact_location_stoch_per_traj = []
    contact_surface_location_nom_per_traj = []
    contact_surface_location_stoch_per_traj = []
    norm_contact_location_deviation_nom_per_traj = []
    norm_contact_location_deviation_stoch_per_traj = []
    for time_idx in range(len(ee_positions_stoch[sim])):
        contact_location_nom_k = []
        contact_location_stoch_k = []
        contact_surface_location_nom_k = []
        contact_surface_location_stoch_k = []
        for contact_idx in range(len(conf.ee_frame_names)):
            # FL
            if contact_idx==0:
                CURR_IN_CONTACT = contacts_logic_N[time_idx][1]
                contact_surface_location_per_foot = contacts_position_N[time_idx][3:6]
            # FR
            elif contact_idx==1:
                CURR_IN_CONTACT = contacts_logic_N[time_idx][0]
                contact_surface_location_per_foot = contacts_position_N[time_idx][0:3]
            # HL
            elif contact_idx==2:
                CURR_IN_CONTACT = contacts_logic_N[time_idx][3]
                contact_surface_location_per_foot = contacts_position_N[time_idx][9:12]
            # HR
            elif contact_idx==3:
                CURR_IN_CONTACT = contacts_logic_N[time_idx][2]
                contact_surface_location_per_foot = contacts_position_N[time_idx][6:9]
            # compute contact location deviation only for the activated contacts
            if CURR_IN_CONTACT:
                contact_surface_location_stoch_k += [contact_surface_location_per_foot]
                contact_location_stoch_k += [ee_positions_stoch[sim][time_idx][contact_idx]]
                if time_idx < len(ee_positions_nom[sim]):
                    contact_surface_location_nom_k += [contact_surface_location_per_foot]
                    contact_location_nom_k += [ee_positions_nom[sim][time_idx][contact_idx]] 
        # since some trajectories fail before resuming till the end in the nominal case
        if time_idx < len(ee_positions_nom[sim]):
            contact_location_nom_per_traj += [contact_location_nom_k]
            contact_surface_location_nom_per_traj += [contact_surface_location_nom_k]
            # compute the contact location at the time of landing
#             if CURR_IN_CONTACT:
            norm_contact_location_deviation_nom_per_traj.append(
                np.linalg.norm(np.asarray(contact_location_nom_k) - np.asarray(contact_surface_location_nom_k))
            )
#             else:
#             norm_contact_location_deviation_nom_per_traj.append(0.)
        contact_location_stoch_per_traj += [contact_location_stoch_k]
        contact_surface_location_stoch_per_traj += [contact_surface_location_stoch_k]
#         if CURR_IN_CONTACT:
        norm_contact_location_deviation_stoch_per_traj.append(
                    np.linalg.norm(np.asarray(contact_location_stoch_k) - np.asarray(contact_surface_location_stoch_k))
                )
#         else:
#         norm_contact_location_deviation_stoch_per_traj.append(0.)
    contact_location_nom_total += [contact_location_nom_per_traj]
    contact_location_stoch_total += [contact_location_stoch_per_traj]
    contact_surface_location_nom_total += [contact_surface_location_nom_per_traj]
    contact_surface_location_stoch_total += [contact_surface_location_stoch_per_traj]
    norm_contact_location_deviation_nom_total += [norm_contact_location_deviation_nom_per_traj]
    norm_contact_location_deviation_stoch_total += [norm_contact_location_deviation_stoch_per_traj]

In [10]:
import matplotlib.pylab as plt

# compute statistics
mean_nom_total = np.zeros(conf.N)
std_nom_total = np.zeros(conf.N)
mean_stoch_total = np.zeros(conf.N)
std_stoch_total = np.zeros(conf.N)
for traj_idx in range(conf.N):
    nb_samples_nom = 0.
    nb_samples_stoch = 0.
    samples_nom_k = []
    samples_stoch_k = []
    for sim in range(nb_sims):
        if traj_idx < len(norm_contact_location_deviation_nom_total[sim]):
            mean_nom_total[traj_idx] += norm_contact_location_deviation_nom_total[sim][traj_idx]
            samples_nom_k += [norm_contact_location_deviation_nom_total[sim][traj_idx]]
            nb_samples_nom += 1
        mean_stoch_total[traj_idx] += norm_contact_location_deviation_stoch_total[sim][traj_idx]
        samples_stoch_k += [norm_contact_location_deviation_stoch_total[sim][traj_idx]]
        nb_samples_stoch += 1
    mean_nom_total[traj_idx] = np.mean(samples_nom_k)
    std_nom_total[traj_idx] = np.std(samples_nom_k)
    mean_stoch_total[traj_idx] = np.mean(samples_stoch_k)
    std_stoch_total[traj_idx] = np.std(samples_stoch_k)

# plot the mean over the horizon length 
fig, ax = plt.subplots(1, 1, sharex=True) 
for sim in range(nb_sims):
    time_nom   = np.arange(0, np.round((len(norm_contact_location_deviation_nom_total[sim]))*conf.dt, 2), conf.dt)
    time_stoch = np.arange(0, np.round((len(norm_contact_location_deviation_stoch_total[sim]))*conf.dt, 2), conf.dt)
plt.plot(time_stoch, mean_stoch_total, color='green')
plt.fill_between(time_stoch, mean_stoch_total+2*std_stoch_total, mean_stoch_total-2*std_stoch_total, color='green', alpha=0.1)
plt.plot(time_stoch, mean_nom_total, color='blue')
plt.fill_between(time_stoch, mean_nom_total+2*std_nom_total, mean_nom_total-2*std_nom_total, color='blue', alpha=0.1)
ax.set_xlabel('Time (s)', fontsize=14)
ax.set_ylabel('norm of contact locations deviations', fontsize=14)
fig.suptitle('Trotting motion')


IndexError: list index out of range